In [ ]:
import numpy as np
import pandas as pd
from nilearn import image, input_data, plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_stat_map
import matplotlib.pyplot as plt
from scipy import stats
import os

Trial information from original 'README' file:

onset(sec) duration(sec) emotion

0 30 calm

30 30 white noise

60 30 afraid

90 30 white noise

120 30 delighted

150 30 white noise

180 30 depressed

210 30 white noise

240 30 excited

270 30 white noise

300 30 delighted

330 30 white noise

360 30 depressed

390 30 white noise

420 30 calm

450 30 white noise

480 30 excited

510 30 white noise

540 30 afraid

570 30 white noise

In [ ]:
# Using Pandas DataFrame method to segment the data.
def create_events_df():
    """Create events DataFrame(Pandas) from the trial segments information"""
    events = pd.DataFrame({
        'onset': [0, 60, 120, 180, 240, 300, 360, 420, 480, 540],
        'duration': [30] * 10,
        'emotion': ['calm', 'afraid', 'delighted', 'depressed', 
                   'excited', 'delighted', 'depressed', 'calm', 
                   'excited', 'afraid']
    })
    # Create trial_type column grouping emotions by valence - into broader groups.
    valence_mapping = {
        'calm': 'neutral',
        'afraid': 'negative',
        'delighted': 'positive',
        'depressed': 'negative',
        'excited': 'positive'
    }
    # map valence type in addition to original emotion categories.
    # Doing so will make data access during analysis easier.
    events['trial_type'] = events['emotion'].map(valence_mapping)
    return events

In [ ]:
def load_and_prepare_data(subject_id):
    """Load and prepare fMRI data for a single subject"""
    # Construct file paths
    func_file = f'sub-{subject_id}/func/sub-{subject_id}_task-fe_bold.nii'
    anat_file = f'sub-{subject_id}/anat/sub-{subject_id}_T1w.nii'
    
    # Load the functional and anatomical images
    func_img = image.load_img(func_file)
    anat_img = image.load_img(anat_file)
    
    return func_img, anat_img

In [ ]:
def create_masks(anat_img):
    """Create ROI masks for emotional processing regions"""
    from nilearn.datasets import fetch_atlas_harvard_oxford
    
    # Fetch both cortical and subcortical Harvard-Oxford atlases
    cortical = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
    subcortical = fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')
    
    # Create masks for regions of interest
    roi_masks = {
        # Subcortical regions
        'amygdala_left': image.math_img('img == 2', img=subcortical.maps),    
        'amygdala_right': image.math_img('img == 3', img=subcortical.maps),   
        'hippocampus_left': image.math_img('img == 4', img=subcortical.maps), 
        'hippocampus_right': image.math_img('img == 5', img=subcortical.maps),
        
        # Cortical regions
        'acc': image.math_img('img == 25', img=cortical.maps),                
        'vmpfc': image.math_img('img == 27', img=cortical.maps),             
        'dlpfc_left': image.math_img('img == 3', img=cortical.maps),         
        'dlpfc_right': image.math_img('img == 4', img=cortical.maps),        
        'parietal_left': image.math_img('np.logical_or(img == 9, img == 10)', img=cortical.maps),  
        'parietal_right': image.math_img('np.logical_or(img == 11, img == 12)', img=cortical.maps) 
    }
    
    # Create bilateral masks
    bilateral_pairs = [
        ('amygdala', 'amygdala_left', 'amygdala_right'),
        ('hippocampus', 'hippocampus_left', 'hippocampus_right'),
        ('dlpfc', 'dlpfc_left', 'dlpfc_right'),
        ('parietal', 'parietal_left', 'parietal_right')
    ]
    
    for name, left, right in bilateral_pairs:
        roi_masks[f'{name}_bilateral'] = image.math_img(
            "img1 + img2", 
            img1=roi_masks[left], 
            img2=roi_masks[right]
        )
    
    return roi_masks, cortical, subcortical

def verify_roi_masks(roi_masks, anat_img, output_dir='roi_verification'):
    """
    Create visualization plots to verify ROI mask locations
    
    Parameters:
    -----------
    roi_masks : dict
        Dictionary containing the ROI mask images
    anat_img : Nifti1Image
        Anatomical reference image
    output_dir : str
        Directory to save the verification plots
    """
    import os
    from nilearn import plotting
    import matplotlib.pyplot as plt
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Set up the MNI template for background
    from nilearn.datasets import load_mni152_template
    template = load_mni152_template()
    
    # Color mapping for different types of ROIs
    colors = {
        'amygdala': 'red',
        'hippocampus': 'blue',
        'acc': 'green',
        'vmpfc': 'purple',
        'dlpfc': 'orange',
        'parietal': 'yellow'
    }
    
    # Create individual ROI plots
    for roi_name, roi_mask in roi_masks.items():
        print(f"Creating visualization for {roi_name}...")
        
        # Create figure with multiple views
        fig = plt.figure(figsize=(15, 5))
        
        # Sagittal view
        ax1 = plt.subplot(131)
        display = plotting.plot_roi(
            roi_mask, 
            template,
            display_mode='x',
            cut_coords=3,
            title=f'{roi_name} - Sagittal',
            axes=ax1
        )
        
        # Coronal view
        ax2 = plt.subplot(132)
        display = plotting.plot_roi(
            roi_mask,
            template,
            display_mode='y',
            cut_coords=3,
            title=f'{roi_name} - Coronal',
            axes=ax2
        )
        
        # Axial view
        ax3 = plt.subplot(133)
        display = plotting.plot_roi(
            roi_mask,
            template,
            display_mode='z',
            cut_coords=3,
            title=f'{roi_name} - Axial',
            axes=ax3
        )
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f'{roi_name}_verification.png'))
        plt.close()
    
    # Create a summary plot with all bilateral ROIs
    print("Creating summary visualization...")
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
    
    # Get all bilateral ROIs
    bilateral_rois = {k: v for k, v in roi_masks.items() if 'bilateral' in k}
    
    # Create multi-ROI display
    displays = []
    for view, ax, title in zip(['x', 'y', 'z'], [ax1, ax2, ax3], 
                              ['Sagittal', 'Coronal', 'Axial']):
        display = plotting.plot_roi(
            template,
            template,
            display_mode=view,
            cut_coords=1,
            title=f'All ROIs - {title}',
            axes=ax
        )
        displays.append(display)
        
        # Add each ROI with a different color
        for roi_name, roi_mask in bilateral_rois.items():
            base_name = roi_name.split('_')[0]
            if base_name in colors:
                display.add_overlay(roi_mask, cmap=plotting.cm.alpha_cmap(colors[base_name]))
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'all_rois_summary.png'))
    plt.close()





def main():
    # Previous main code...
    
    
    
    # Rest of the analysis pipeline...


In [ ]:
def single_subject_analysis(subject_id, events_df, roi_masks):
    """Perform first-level analysis for a single subject"""
    # Load subject data
    func_img, anat_img = load_and_prepare_data(subject_id)
    
    # Initialize FirstLevelModel
    model = FirstLevelModel(
        t_r=2.0,  # Replace with actual TR
        noise_model='ar1',
        standardize=True,
        hrf_model='spm',
        drift_model='cosine'
    )
    
    # Fit the GLM
    model.fit(func_img, events_df)
    
    # Create contrasts
    contrasts = {
        'positive_vs_neutral': 'positive - neutral',
        'negative_vs_neutral': 'negative - neutral',
        'positive_vs_negative': 'positive - negative'
    }
    
    # Compute contrast maps
    contrast_maps = {}
    for contrast_id, contrast_def in contrasts.items():
        contrast_maps[contrast_id] = model.compute_contrast(contrast_def)
    
    # Extract ROI signals
    roi_signals = {}
    for roi_name, roi_mask in roi_masks.items():
        masker = input_data.NiftiMasker(mask_img=roi_mask)
        roi_signals[roi_name] = masker.fit_transform(func_img)
    
    return contrast_maps, roi_signals

def group_analysis(all_subject_contrasts):
    """Perform group-level analysis"""
    from nilearn.glm import second_level
    
    # Initialize second-level model
    group_model = second_level.SecondLevelModel()
    
    # Perform group analysis for each contrast
    group_results = {}
    for contrast_id in all_subject_contrasts[0].keys():
        contrast_maps = [subj_contrasts[contrast_id] 
                        for subj_contrasts in all_subject_contrasts]
        group_results[contrast_id] = group_model.fit(contrast_maps).compute_contrast()
    
    return group_results

def plot_results(group_results, output_dir):
    """Plot and save group-level results"""
    for contrast_id, stat_map in group_results.items():
        display = plotting.plot_stat_map(
            stat_map,
            threshold=3.0,  # Adjust threshold as needed
            title=contrast_id
        )
        plt.savefig(f'{output_dir}/{contrast_id}_group_map.png')
        plt.close()

def main():
    # Parameters
    n_subjects = 40
    output_dir = 'results'
    os.makedirs(output_dir, exist_ok=True)
    
    # Create events DataFrame
    events_df = create_events_df()
    
    # Initialize storage for all subjects' results
    all_subject_contrasts = []
    all_subject_roi_signals = []
    
    # Add ROI verification after creating masks
    roi_masks, cortical, subcortical = create_masks(anat_img)
    verify_roi_masks(roi_masks, anat_img)
    
    # Single subject analysis
    for subject_id in range(1, n_subjects + 1):
        subject_id_str = f'{subject_id:02d}'
        print(f'Processing subject {subject_id_str}...')
        
        contrasts, roi_signals = single_subject_analysis(
            subject_id_str, events_df, roi_masks
        )
        
        all_subject_contrasts.append(contrasts)
        all_subject_roi_signals.append(roi_signals)
    
    # Group analysis
    group_results = group_analysis(all_subject_contrasts)
    
    # Plot results
    plot_results(group_results, output_dir)
    
    # Save results
    # Add code to save numerical results, statistics, etc.

if __name__ == '__main__':
    main()